In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

# 필요한 함수 정의
def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)


PATH = './data/'

In [2]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
display(train_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [3]:
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

In [4]:
## days의 변수로 만들기
train_err['datetime'] = train_err['time'].apply(make_datetime)
min_day = train_err.datetime.min().date()

train_err['days'] = train_err.loc[:,'datetime'].dt.date - min_day
train_err['days'] = train_err['days'].dt.days
display(train_err)

,user_id,time,model_nm,fwver,errtype,errcode,datetime,days
0,10000,20201101025616,model_3,05.15.2138,15,1,2020-11-01 02:00:00,1
1,10000,20201101030309,model_3,05.15.2138,12,1,2020-11-01 03:00:00,1
2,10000,20201101030309,model_3,05.15.2138,11,1,2020-11-01 03:00:00,1
3,10000,20201101050514,model_3,05.15.2138,16,1,2020-11-01 05:00:00,1
4,10000,20201101050515,model_3,05.15.2138,4,0,2020-11-01 05:00:00,1
...,...,...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1,2020-11-30 16:00:00,30
16554659,24999,20201130172625,model_3,05.15.2138,16,1,2020-11-30 17:00:00,30
16554660,24999,20201130172625,model_3,05.15.2138,4,0,2020-11-30 17:00:00,30
16554661,24999,20201130172631,model_3,05.15.2138,4,0,2020-11-30 17:00:00,30


In [5]:
##시간별 데이터 만드는 코딩
def time_encoding(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    day   = int(x[6:8])
    hour  = int(x[8:10])
    y = day*100 + hour
    for i in range(30):
        if y == (i+1)*100:
            return 0
        if y == (i+1)*100 + 1:
            return 1
        if y == (i+1)*100 + 2:
            return 2
        if y == (i+1)*100 + 3:
            return 3
        if y == (i+1)*100 + 4:
            return 4
        if y == (i+1)*100 + 5:
            return 5
        if y == (i+1)*100 + 6:
            return 6
        if y == (i+1)*100 + 7:
            return 7
        if y == (i+1)*100 + 8:
            return 8
        if y == (i+1)*100 + 9:
            return 9
        if y == (i+1)*100 + 10:
            return 10
        if y == (i+1)*100 + 11:
            return 11
        if y == (i+1)*100 + 12:
            return 12
        if y == (i+1)*100 + 13:
            return 13
        if y == (i+1)*100 + 14:
            return 14
        if y == (i+1)*100 + 15:
            return 15
        if y == (i+1)*100 + 16:
            return 16
        if y == (i+1)*100 + 17:
            return 17
        if y == (i+1)*100 + 18:
            return 18
        if y == (i+1)*100 + 19:
            return 19
        if y == (i+1)*100 + 20:
            return 20
        if y == (i+1)*100 + 21:
            return 21
        if y == (i+1)*100 + 22:
            return 22
        if y == (i+1)*100 + 23:
            return 23
        if day == 31: # 10월 31일 23:59의 경우
            return 0

In [ ]:
#적용하기
train_err['timeslot'] = train_err['time'].apply(time_encoding)

In [6]:
train_err[train_err['time']==20201031235959] ## 1031일 확인하기
#timeslot 추가분 /// 31변수가 NAN으로 입력됨
#train_err[train_err['timeslot'].isnull()==True]

## 날짜 데이터는 1031 ~ 1202 까지 존재

,user_id,time,model_nm,fwver,errtype,errcode,datetime,days,timeslot
6409135,15801,20201031235959,model_0,04.22.1750,16,1,2020-10-31 23:00:00,0,0
9204586,18418,20201031235959,model_0,04.22.1750,26,1,2020-10-31 23:00:00,0,0
12134438,20974,20201031235959,model_0,04.22.1750,26,1,2020-10-31 23:00:00,0,0
13424899,22249,20201031235959,model_0,04.22.1750,26,1,2020-10-31 23:00:00,0,0
13585684,22385,20201031235959,model_2,04.33.1185,26,1,2020-10-31 23:00:00,0,0


In [7]:
train_err

,user_id,time,model_nm,fwver,errtype,errcode,datetime,days,timeslot
0,10000,20201101025616,model_3,05.15.2138,15,1,2020-11-01 02:00:00,1,2
1,10000,20201101030309,model_3,05.15.2138,12,1,2020-11-01 03:00:00,1,3
2,10000,20201101030309,model_3,05.15.2138,11,1,2020-11-01 03:00:00,1,3
3,10000,20201101050514,model_3,05.15.2138,16,1,2020-11-01 05:00:00,1,5
4,10000,20201101050515,model_3,05.15.2138,4,0,2020-11-01 05:00:00,1,5
...,...,...,...,...,...,...,...,...,...
16554658,24999,20201130163051,model_3,05.15.2138,15,1,2020-11-30 16:00:00,30,16
16554659,24999,20201130172625,model_3,05.15.2138,16,1,2020-11-30 17:00:00,30,17
16554660,24999,20201130172625,model_3,05.15.2138,4,0,2020-11-30 17:00:00,30,17
16554661,24999,20201130172631,model_3,05.15.2138,4,0,2020-11-30 17:00:00,30,17


In [8]:
train_err=train_err.drop(columns=['datetime','time']) ## 시간처리 완료로 시간 데이터 드랍
#train_errf2 = train_errf.drop(columns=['user_id','model_nm']) #ID 드랍하고 모델넘버는 펌웨어 버전과 연관이 있기에 드랍

In [11]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


# 숫자 값으로 변환하기 위해 LabelEncoder로 먼저 변환한다.
encoder = LabelEncoder()
encoder.fit(train_err['fwver'].astype('|S80'))
labels_fwver = encoder.transform(train_err['fwver'].astype('|S80'))
train_err = pd.concat([train_err,pd.Series(labels_fwver)],axis=1)
train_err.rename(columns = {0 : 'labels_fwver'}, inplace = True)

In [12]:
fwver_label = train_err[['user_id','labels_fwver']].values
fwver_check = np.zeros((train_user_number,len(train_err['labels_fwver'].unique())))

for person_idx, ch in tqdm(fwver_label):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    fwver_check[person_idx - train_user_id_min,ch - 1] += 1
fwver_check.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:53<00:00, 312341.54it/s]


(15000, 37)

In [13]:
days_label = train_err[['user_id','days']].values
days_check = np.zeros((train_user_number,len(train_err['days'].unique())))

for person_idx, ch in tqdm(days_label):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    days_check[person_idx - train_user_id_min,ch - 1] += 1
days_check.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:52<00:00, 317842.31it/s]


(15000, 33)

In [46]:
timeslot_label = train_err[['user_id','timeslot']].values
timeslot_check = np.zeros((train_user_number,len(train_err['timeslot'].unique())))

for person_idx, ch in tqdm(timeslot_label):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    timeslot_check[person_idx - train_user_id_min,ch - 1] += 1
timeslot_check.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:51<00:00, 321577.26it/s]


(15000, 24)

In [15]:
id_error = train_err[['user_id','errtype']].values
error = np.zeros((train_user_number,42))

for person_idx, err in tqdm(id_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error[person_idx - train_user_id_min,err - 1] += 1
error.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:57<00:00, 290391.65it/s]


(15000, 42)

In [47]:
x_concat1 = np.concatenate((fwver_check, days_check), axis=1) ## 펌웨어 버전 + 날짜 
x_concat2 = np.concatenate((x_concat1, timeslot_check), axis=1) ## 펌웨어 버전 + 날짜 +시간 
x_concat3 = np.concatenate((x_concat2, error), axis=1) ## 펌웨어 버전 + 날짜 +시간 + 에러

array([[  0.,   0.,   8., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 113.,  56.,   1.],
       [  0.,   0.,   2., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  58.,   8.,   5.],
       [  0.,   0.,   0., ...,   6.,   0.,   0.],
       [  0.,   0.,   4., ...,   0.,   0.,   0.]])

In [17]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)
# error와 동일한 방법으로 person_idx - 10000 위치에 
# person_idx의 problem이 한 번이라도 발생했다면 1
# 없다면 0
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

In [48]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = x_concat3
train_y = problem

print(train_x.shape)
print(train_y.shape)

(15000, 136)
(15000,)


In [49]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'task' : 'train',
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'learning_rate' : 0.01,
                'max_bin' : 255,
                'feature_fraction' : 0.8,
                'min_data_in_leaf'     : 10,
                'num_threads'    : 4, 
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 10 Kfold cross validation
k_fold = KFold(n_splits=10, shuffle=True, random_state=150)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 100
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 4543, number of negative: 8957
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008887 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 20684
[LightGBM] [Info] Number of data points in the train set: 13500, number of used features: 115
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.336519 -> initscore=-0.678848
[LightGBM] [Info] Start training from score -0.678848
Training until validation scores don't improve for 100 rounds
[20]	valid_0's auc: 0.799034	valid_0's pr_auc: 0.701894
[40]	valid_0's auc: 0.802715	valid_0's pr_auc: 0.709187
[60]	valid_0's auc: 0.805149	valid_0's pr_auc: 0.711546
[80]	valid_0's auc: 0.805797	valid_0's pr_auc: 0.711752
[100]	valid_0's auc: 0.806544	valid_0's pr_auc: 0.711828
[120]	valid_0's auc: 0.806655	valid_0's pr_auc: 0.713807
[140]	valid_0's auc: 0.807387	valid_0's pr_auc: 0.714778
[160]	valid_0's auc: 0.807841	valid_0's pr_auc

In [50]:
print(np.mean(auc_scores))

0.8117300029309756


In [51]:
# 숫자 값으로 변환하기 위해 LabelEncoder로 먼저 변환한다.

encoder.fit(train_err['errcode'].astype('|S80'))
errcode_label = encoder.transform(train_err['errcode'].astype('|S80'))
train_err = pd.concat([train_err,pd.Series(errcode_label)],axis=1)
train_err.rename(columns = {0 : 'errcode_label'}, inplace = True)

In [52]:
errcode_label_ = train_err[['user_id','errcode_label']].values
errcode_check = np.zeros((train_user_number,len(train_err['errcode_label'].unique())))

for person_idx, ch in tqdm(errcode_label_):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    errcode_check[person_idx - train_user_id_min,ch - 1] += 1
errcode_check.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:52<00:00, 313485.49it/s]


(15000, 2806)

In [53]:
# errcode를 모델에 적용
x_concat4 = np.concatenate((x_concat3, errcode_check), axis=1) ## 펌웨어 버전 + 날짜 +시간 + 에러 + 에러코드


In [81]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = x_concat4
train_y = problem
print(train_x.shape)
print(train_y.shape)

(15000, 2942)
(15000,)


In [75]:
# Train
#-------------------------------------------------------------------------------------
# validation auc score를 확인하기 위해 정의
def f_pr_auc(probas_pred, y_true):
    labels=y_true.get_label()
    p, r, _ = precision_recall_curve(labels, probas_pred)
    score=auc(r,p) 
    return "pr_auc", score, True
#-------------------------------------------------------------------------------------
models     = []
recalls    = []
precisions = []
auc_scores   = []
threshold = 0.5
# 파라미터 설정
params =      {
                'task' : 'train',
                'boosting_type' : 'gbdt',
                'objective'     : 'binary',
                'metric'        : 'auc',
                'learning_rate' : 0.01,
                'max_bin' : 255,
                'feature_fraction' : 0.8,
                'min_data_in_leaf'     : 10,
                'num_threads'    : 4, 
                'seed': 1015
                }
#-------------------------------------------------------------------------------------
# 10 Kfold cross validation
k_fold = KFold(n_splits=5, shuffle=True, random_state=150)
for train_idx, val_idx in k_fold.split(train_x):

    # split train, validation set
    X = train_x[train_idx]
    y = train_y[train_idx]
    valid_x = train_x[val_idx]
    valid_y = train_y[val_idx]

    d_train= lgb.Dataset(X, y)
    d_val  = lgb.Dataset(valid_x, valid_y)
    
    #run traning
    model = lgb.train(
                        params,
                        train_set       = d_train,
                        num_boost_round = 1000,
                        valid_sets      = d_val,
                        feval           = f_pr_auc,
                        verbose_eval    = 20, 
                        early_stopping_rounds = 100
                       )
    
    # cal valid prediction
    valid_prob = model.predict(valid_x)
    valid_pred = np.where(valid_prob > threshold, 1, 0)
    
    # cal scores
    recall    = recall_score(    valid_y, valid_pred)
    precision = precision_score( valid_y, valid_pred)
    auc_score = roc_auc_score(   valid_y, valid_prob)

    # append scores
    models.append(model)
    recalls.append(recall)
    precisions.append(precision)
    auc_scores.append(auc_score)

    print('==========================================================')

[LightGBM] [Info] Number of positive: 4064, number of negative: 7936
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015591 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 24493
[LightGBM] [Info] Number of data points in the train set: 12000, number of used features: 195
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.338667 -> initscore=-0.669242
[LightGBM] [Info] Start training from score -0.669242
Training until validation scores don't improve for 100 rounds
[20]	valid_0's auc: 0.810079	valid_0's pr_auc: 0.716911
[40]	valid_0's auc: 0.813687	valid_0's pr_auc: 0.717379
[60]	valid_0's auc: 0.815286	valid_0's pr_auc: 0.716607
[80]	valid_0's auc: 0.81619	valid_0's pr_auc: 0.717411
[100]	valid_0's auc: 0.817216	valid_0's pr_auc: 0.719564
[120]	valid_0's auc: 0.8186	valid_0's pr_auc: 0.723667
[140]	valid_0's auc: 0.820044	valid_0's pr_auc: 0.724825
[160]	valid_0's auc: 0.820755	valid_0's pr_auc: 0

In [92]:
print(np.mean(auc_scores))

0.8180412836811944


In [23]:
from pycaret.classification import *

In [84]:
X['problem'] = problem

In [83]:
X = pd.DataFrame(x_concat4)
y = pd.DataFrame(problem)

In [85]:
clf = setup(data = X, target = 'problem') 

,Description,Value
0,session_id,8589
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 2943)"
5,Missing Values,False
6,Numeric Features,771
7,Categorical Features,2171
8,Ordinal Features,False
9,High Cardinality Features,False


In [86]:
best_5 = compare_models(sort = 'Accuracy', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.7925,0.8198,0.5023,0.7979,0.6162,0.4840,0.5085,37.7620
gbc,Gradient Boosting Classifier,0.7864,0.8127,0.4774,0.7987,0.5973,0.4643,0.4929,7.6080
lightgbm,Light Gradient Boosting Machine,0.7838,0.8143,0.5055,0.7641,0.6080,0.4673,0.4865,0.6700
xgboost,Extreme Gradient Boosting,0.7767,0.7971,0.5261,0.7267,0.6101,0.4593,0.4712,14.1640
ada,Ada Boost Classifier,0.7722,0.7950,0.4788,0.7440,0.5821,0.4357,0.4559,1.9850
et,Extra Trees Classifier,0.7719,0.7926,0.4275,0.7890,0.5540,0.4187,0.4543,3.9490
rf,Random Forest Classifier,0.7677,0.7953,0.4180,0.7807,0.5442,0.4071,0.4429,2.0010
ridge,Ridge Classifier,0.7441,0.0000,0.3423,0.7531,0.4701,0.3315,0.3765,5.1250
lda,Linear Discriminant Analysis,0.7428,0.7173,0.3558,0.7331,0.4786,0.3342,0.3729,6.5160
lr,Logistic Regression,0.7289,0.7426,0.3240,0.6961,0.4414,0.2934,0.3308,13.1530


In [93]:
blended = blend_models(estimator_list = best_5, fold = 10, method = 'soft')

IntProgress(value=0, description='Processing: ', max=6)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,00:55:41
Status,. . . . . . . . . . . . . . . . . .,Fitting 7 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Voting Classifier


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker.


In [ ]:
pred_holdout = predict_model(blended)

In [89]:
final_model = finalize_model(blended)

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 625.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 625.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  11.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.]])

In [42]:
id_error = test_err[['user_id','errtype']].values
test_x = np.zeros((14999,42))
for person_idx, err in tqdm(id_error):
    test_x[person_idx - 30000,err - 1] += 1
test_x = test_x.reshape(test_x.shape[0],-1)
test = pd.DataFrame(data=test_x)

NameError: name 'test_err' is not defined

In [43]:
test_err = pd.read_csv(PATH+'test_err_data.csv')

In [44]:
## days의 변수로 만들기
test_err['datetime'] = test_err['time'].apply(make_datetime)
min_day = test_err.datetime.min().date()

test_err['days'] = test_err.loc[:,'datetime'].dt.date - min_day
test_err['days'] = test_err['days'].dt.days
display(test_err)

,user_id,time,model_nm,fwver,errtype,errcode,datetime,days
0,30000,20201101030227,model_1,04.16.3553,31,1,2020-11-01 03:00:00,1
1,30000,20201101030227,model_1,04.16.3553,33,2,2020-11-01 03:00:00,1
2,30000,20201101030228,model_1,04.16.3553,15,1,2020-11-01 03:00:00,1
3,30000,20201101030256,model_1,04.16.3553,22,1,2020-11-01 03:00:00,1
4,30000,20201101030300,model_1,04.16.3553,11,1,2020-11-01 03:00:00,1
...,...,...,...,...,...,...,...,...
16532643,44998,20201130210050,model_1,04.16.3553,40,0,2020-11-30 21:00:00,30
16532644,44998,20201130211831,model_1,04.16.3553,31,1,2020-11-30 21:00:00,30
16532645,44998,20201130211832,model_1,04.16.3553,15,1,2020-11-30 21:00:00,30
16532646,44998,20201130212259,model_1,04.16.3553,16,1,2020-11-30 21:00:00,30


In [45]:
#적용하기
test_err['timeslot'] = test_err['time'].apply(time_encoding)

In [94]:
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [95]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder


# 숫자 값으로 변환하기 위해 LabelEncoder로 먼저 변환한다.
encoder = LabelEncoder()
encoder.fit(test_err['fwver'].astype('|S80'))
labels_fwver = encoder.transform(test_err['fwver'].astype('|S80'))
test_err = pd.concat([test_err,pd.Series(labels_fwver)],axis=1)
test_err.rename(columns = {0 : 'labels_fwver'}, inplace = True)

In [96]:
fwver_label_test = test_err[['user_id','labels_fwver']].values
fwver_check_test = np.zeros((test_user_number,len(test_err['labels_fwver'].unique())))

for person_idx, ch in tqdm(fwver_label_test):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    fwver_check_test[person_idx - test_user_id_min,ch - 1] += 1
fwver_check_test.shape

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [00:52<00:00, 316284.41it/s]


(14999, 40)

In [112]:
days_label_test = test_err[['user_id','days']].values
days_check_test = np.zeros((test_user_number,len(test_err['days'].unique())))

for person_idx, ch in tqdm(days_label_test):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    days_check_test[person_idx - test_user_id_min,ch-2] += 1
days_check_test.shape

  2%|█                                                                   | 250040/16532648 [00:00<00:46, 352590.50it/s]


IndexError: index 41 is out of bounds for axis 1 with size 39

In [ ]:
timeslot_label_test = test_err[['user_id','timeslot']].values
timeslot_check_test = np.zeros((test_user_number,len(test_err['timeslot'].unique())))

for person_idx, ch in tqdm(timeslot_label_test):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    timeslot_check_test[person_idx - test_user_id_min,ch - 1] += 1
timeslot_check_test.shape

In [ ]:
error_label_test = train_err[['user_id','errtype']].values
error_check_test = np.zeros((test_user_number,len(test_err['errtype'].unique())))

for person_idx, err in tqdm(error_label_test):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    error_check_test[person_idx - test_user_id_min,err - 1] += 1
error_check_test.shape

In [ ]:
x_test1 = np.concatenate((fwver_check_test, days_check_test), axis=1) ## 펌웨어 버전 + 날짜 
x_test2 = np.concatenate((x_concat1, timeslot_check_test), axis=1) ## 펌웨어 버전 + 날짜 +시간 
x_test3 = np.concatenate((x_concat2, error_check_test), axis=1) ## 펌웨어 버전 + 날짜 +시간 + 에러

In [ ]:
#숫자 값으로 변환하기 위해 LabelEncoder로 먼저 변환한다.

encoder.fit(test_err['errcode'].astype('|S80'))
errcode_label = encoder.transform(test_err['errcode'].astype('|S80'))
test_err = pd.concat([test_err,pd.Series(errcode_label)],axis=1)
test_err.rename(columns = {0 : 'errcode_label'}, inplace = True)

In [ ]:
errcode_label_test = test_err[['user_id','errcode_label']].values
errcode_check_test = np.zeros((test_user_number,len(test_err['errcode_label'].unique())))

for person_idx, ch in tqdm(errcode_label_test):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    errcode_check_test[person_idx - test_user_id_min,ch - 1] += 1
errcode_check_test.shape

In [ ]:
# errcode를 모델에 적용
x_test4 = np.concatenate((x_test3, errcode_check_test), axis=1) ## 펌웨어 버전 + 날짜 +시간 + 에러 + 에러코드

In [108]:
test_err[test_err['days']==41]

,user_id,time,model_nm,fwver,errtype,errcode,datetime,days,timeslot,labels_fwver
250040,30263,20201211203507,model_2,04.33.1261,6,14,2020-12-11 20:00:00,41,20,21


In [111]:
len(test_err['days'].unique())

39

In [113]:
test_err[['user_id','days']].values

array([[30000,     1],
       [30000,     1],
       [30000,     1],
       ...,
       [44998,    30],
       [44998,    30],
       [44998,    30]], dtype=int64)

39